In [32]:
spark

Waiting for a Spark session to start...

org.apache.spark.sql.SparkSession@1af47cf8

In [33]:
spark.version

2.4.7

In [ ]:
import org.apache.spark.sql.functions.col

import org.apache.spark.sql.types.DoubleType
import org.apache.spark.ml.classification.{RandomForestClassificationModel, RandomForestClassifier}
import org.apache.spark.mllib.tree.configuration.BoostingStrategy
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.types._
import org.apache.spark.ml.linalg.{DenseVector, SparseVector, Vector, Vectors}
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

In [ ]:
var train = spark.read
                    .option("delimiter", ",")
                    .option("header", "True")
                    .option("encoding","utf-8")
                    .csv("/labs/slaba05/lab05_train.csv")
var test = spark.read
                    .option("delimiter", ",")
                    .option("header", "True")
                    .option("encoding","utf-8")
                    .csv("/labs/slaba05/lab05_test.csv")

In [ ]:
var feature_list = Array[String]()
for (t <- train.columns if (t!="ID"))
{ train = train.withColumn(t, col(t).cast(FloatType))
        feature_list +:= t}
val b = feature_list.filter(! _.contains("TARGET"))

In [ ]:
for (t <- test.columns if (t!="ID"))
{ test = test.withColumn(t, col(t).cast(FloatType))}

train = train.withColumn("TARGET", col("TARGET").cast(IntegerType))

In [ ]:
train = train.na.drop()
test = test.na.drop()

In [ ]:
val assembler = new VectorAssembler()
                    .setInputCols(b)
                    .setOutputCol("features")


val evaluator = new BinaryClassificationEvaluator()
                        .setLabelCol("TARGET")
                        .setRawPredictionCol("probability")
                        .setMetricName("areaUnderROC")


val model = new RandomForestClassifier()
      .setFeaturesCol("features")
      .setLabelCol("TARGET")
      .setNumTrees(100)
      .setMaxDepth(8)
      .setSubsamplingRate(0.75)

In [ ]:
val train_data = assembler.transform(train)
val test_data = assembler.transform(test)

In [ ]:
val rf_model = model.fit(train_data)

In [ ]:
val pred_train = rf_model.transform(train_data)
val pred_test = model_rf.transform(test_data)

In [ ]:
val accuracy = evaluator.evaluate(pred_train)

In [ ]:
val sparse_to_dense = udf((v: Vector) =>  v.toArray)

val res = pred_test.select(col("ID").alias("id"), sparse_to_dense(col("probability")).getItem(1).alias("target")).coalesce(1)

In [ ]:
res.write.mode("overwrite").format("csv").option("delimiter", "\t").option("header", "True").option("encoding", "UTF-8").save("lab05")

In [34]:
spark.stop()